In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import performance_benchmark as pbench

# Intro

There are a number of python packages to work with FCA. In this notebook we will compare their performances in the basic FCA task: constructing the concept lattice from a formal context.

We consider two packages: FCApy and Concepts

// More packages can be compared in the future

# Install competitors libraries

`FCApy` package (by Egor Dudyrev, HSE Moscow): https://github.com/EgorDudyrev/FCApy 

In [ ]:
from fcapy import LIB_INSTALLED
from fcapy.context import FormalContext
from fcapy.lattice import ConceptLattice
from fcapy.visualizer import ConceptLatticeVisualizer

`Concepts` package (by Sebastian Bank, University of Leipzig): https://github.com/xflr6/concepts

In [ ]:
import concepts

`fcapsy` package (by Tomáš Mikula, Palacký University): https://github.com/mikulatomas/fcapsy

Upd. We drop `fcapsy` package from the benchmark since now it uses `concepts` package under the hood

# Load data

First we load some classic FCA contexts (datasets)

In [ ]:
contexts_to_test = ['animal_movement', 'digits', 'gewaesser','lattice', 'liveinwater', 'tealady']
frames_classic = pbench.load_classic_context(contexts_to_test)

Add Bob-Ross dataset which has more objects and attributes than the classic FCA datasets

In [ ]:
frames_classic['bob_ross'] = pbench.load_bob_ross_dataframe()

These classic real world contexts are small so we add some big random contexts to our examination

In [ ]:
n_objects_vars = [10, 30, 100]
n_attributes_vars = [10, 30, 50]
densities_vars = [0.1, 0.5, 0.9]

frames_random = pbench.generate_random_contexts(n_objects_vars, n_attributes_vars, densities_vars)

In [ ]:
frames = dict(frames_classic, **frames_random)
#frames = dict(frames_classic)

# Run benchmarks

## Default lattice visualizations

Let us take one classic FCA context 'animal movement' and a bigger one 'bob ross' dataset

The description of Animals context:
* objects (rows) are Animals
* attributes (columns) are Actions
* the table shows whether an Animal can perform an Action

The description of Bob Ross dataset:
* objects (rows) are paintings by Bob Ross
* attributes (columns) are specific elements in these paintings
* the table shows whether an element is on a painting

In [ ]:
K_names = ['animal_movement', 'tealady']#'bob_ross']

### Visualization by `concepts`

In [ ]:
from datetime import datetime

In [ ]:
pbench.visualize_by_concepts(K_names, frames, "imgs/lattice_visualization/concepts")

### Visualization by `fcapy`

In [ ]:
pbench.visualize_by_fcapy(K_names, frames, "imgs/lattice_visualization/fcapy")

## Time to construct a lattice

Run the benchmarks

In [ ]:
n_runs = 10
timeout_secs = 5*60

In [ ]:
frames_order = sorted(frames, key=lambda K_name: pbench.get_context_stat(frames[K_name])['n_connections'])

In [ ]:
ctx_names_vals = frames_order[:3]
lib_names_vals = ['concepts', 'fcapy']#, 'fcapsy']
n_runs*len(ctx_names_vals)*len(lib_names_vals)

In [ ]:
%%time
stats_df = pbench.compute_stats(ctx_names_vals, lib_names_vals, n_runs, timeout_secs, frames)

In [ ]:
stats_df.to_csv('benchmark_stats.csv')

# Analyze the results

In [ ]:
import pandas as pd

In [ ]:
stats_df = pd.read_csv('benchmark_stats.csv', index_col=0)
print(stats_df.shape)
stats_df.head()

In [ ]:
stats_df = stats_df.fillna(timeout_secs)

In [ ]:
context_stat_feats = ['n_objects', 'n_attributes', 'n_connections', 'density']

In [ ]:
pbench.save_context_stats(stats_df, context_stat_feats)

In [ ]:
pbench.save_lattice_time_plot('imgs/lattice_construction_time/classic_contexts.png', context_stat_feats, stats_df[~stats_df['is_random']], timeout_secs)
pbench.save_lattice_time_plot('imgs/lattice_construction_time/random_contexts.png', context_stat_feats, stats_df[stats_df['is_random']], timeout_secs)

In [ ]:
pbench.save_extent_intent_time_plot('imgs/intent_extent_time/all_data.png', context_stat_feats, stats_df)